# SCIPY minimize

Below will test if minimize functionality is better suited for the nature of our GSIB contraint

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [3]:
# Set options to print all columns of the dataframe

pd.set_option('display.max_columns', None)

In [30]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=40, usecols='A:AO')

In [25]:
df.head()

,Product,start,grow,shrink,spread,A_L,b1_leverage,a_rwa,s_rwa,gsib_leverage,gsib_xjd_claim,gsib_xjd_liab,gsib_intrafin_claim,gsib_intrafin_liab,gsib_securities,gsib_payment,gsib_auc,gsib_underwriting,gsib_otc,`,gsib_level3,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,gsib_leverage_score,gsib_xjd_claim_score,gsib_xjd_liab_score,gsib_intrafin_claim_score,gsib_intrafin_liab_score,gsib_securities_score,gsib_payment_score,gsib_auc_score,gsib_underwriting_score,gsib_otc_score,gsib_trading_score,gsib_level3_score,total_score,cet1_contr
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,1,0.2,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144
2,subprime_auto,4000,2000,-1000,150,1,1,0.8,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.8,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.8,0.000039
3,mtg_30_fixed,50000,30000,-15000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,10.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,10.0,0.000482
4,mtg_15_fixed,15000,8000,-3000,72,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144
5,mtg_7_fixed,5000,2500,-2000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,1.0,0.000048


In [26]:
# Read in constraints

constraints = pd.read_excel('./data/sample_bs.xlsx', index_col=0, sheet_name='constraints')

constraints

,CET1,T1,total_capital,TLAC
SRWA,0.080000,0.095,0.115,0.195
ARWA,0.080000,0.095,0.115,0.195
leverage,0.075000,0.090,NaN,NaN
GSIB,0.000048,NaN,NaN,NaN


## Objective function

- This is simply an array of the negative return (i.e., profitability)
- Since the optimization will actually be minimizing this, we just flip the sign

In [86]:
# Define NSI to be passed as part of args

nsi = np.array(df['spread'])

In [87]:
def obj_fun(x, nsi):
    sva = 0
#     nsi = np.array(df['spread'])
    
    for i in range(len(x)):
        sva += -(nsi[i] * x[i])
    
    return sva

## Starting Guess

- The initial balances

In [34]:
x0 = np.array(df['start'])

## Inequality Constraints
- Advanced and Standardized RWA

In [146]:
sRWA_mins = list(constraints.loc['SRWA', :])

In [147]:
sRWA_mins[0]

0.08

In [167]:
srwa_cet1(np.array(df['start']))

308.9999999999927

In [166]:
def srwa_cet1(x, df=df):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    sRWA_mins = list(constraints.loc['SRWA', :])
    
    s_rwa = np.array(df['s_rwa'])
    cet1_resource = np.array(df['CET1_resource'])
    cet1_constraint = 0
    
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    for i in range(len(x)):
        cet1_constraint += -((sRWA_mins[0] + gsib_addon) * s_rwa[i] * x[i]) + cet1_resource[i] * x[i]
        
    return cet1_constraint

## Equality Constraints

- Assets must equal liabilities

In [149]:
def eq_constraint(x, df=df):   
    # Create array of Asset / Liability weights
    a_l_weight = np.array(df['A_L'])
    
    # Define target (assets must equal liabilities so the difference is 0)
    eq = 0
    
    for i in range(len(x)):
        eq = eq - x[i] * a_l_weight[i]
    
    return eq

## Consolidate Constraints

Convert to Scipy minimize formatting

In [168]:
con1 = {'type': 'ineq', 'fun': srwa_cet1}
con2 = {'type': 'eq', 'fun': eq_constraint}

cons = [con1, con2]

## Upper and Lower Bounds

In [169]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [170]:
# Define bounds in format needed for linprog

bounds = tuple(df['bounds'])

In [171]:
bounds

((10000, 23000),
 (3000, 6000),
 (35000, 80000),
 (12000, 23000),
 (3000, 7500),
 (16000, 30000),
 (8000, 13500),
 (50000, 80000),
 (13000, 17500),
 (15000, 35000),
 (46500, 70000),
 (28000, 60000),
 (85000, 140000),
 (70000, 105000),
 (48000, 75000),
 (65000, 95000),
 (27500, 34000),
 (9700, 13000),
 (750, 1500),
 (0, 0),
 (140000, 240000),
 (53500, 88500),
 (65000, 110000),
 (15000, 26000),
 (12000, 30000),
 (3500, 7000),
 (30000, 75000),
 (17000, 32000),
 (12000, 22000),
 (6500, 12500),
 (16000, 40000),
 (9500, 16000),
 (5000, 10000),
 (2000, 4000),
 (3000, 1053000),
 (0, 0),
 (1500, 154000),
 (1000, 21000),
 (1500, 31000),
 (3000, 103000))

## Minimize

Solvers:
- trust-constr
- SLSQP

In [172]:
sol = spo.minimize(obj_fun, x0, method = 'trust-constr', bounds=bounds, constraints=cons, args = (nsi))

In [173]:
sol.success

True

In [174]:
sol.fun

-159315187.11519355

In [176]:
sol.x

array([ 1.00000000e+04,  6.00000000e+03,  8.00000000e+04,  2.30000000e+04,
        7.49999999e+03,  3.00000000e+04,  1.35000000e+04,  8.00000000e+04,
        1.75000000e+04,  1.50000000e+04,  4.65000000e+04,  2.80000000e+04,
        8.50000000e+04,  1.05000000e+05,  7.50000000e+04,  9.50000000e+04,
        3.40000000e+04,  1.30000000e+04,  1.50000000e+03, -1.03756147e-15,
        2.40000000e+05,  8.85000000e+04,  1.10000000e+05,  2.60000000e+04,
        3.00000000e+04,  4.05067364e+03,  7.50000000e+04,  3.20000000e+04,
        2.20000000e+04,  1.25000000e+04,  4.00000000e+04,  9.50000001e+03,
        5.00000002e+03,  2.00000001e+03,  3.00000001e+03, -6.07532043e-16,
        6.04493264e+04,  1.00000000e+03,  1.50000000e+03,  3.00000000e+03])

In [177]:
test = pd.DataFrame(sol.x)

In [178]:
test.to_excel('./results/scr.xlsx')